In [ ]:
import os
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
warnings.filterwarnings("ignore")

label_methods = [
    "atr",
    "trend",
    "trend_profit",
    "trend_multi",
    "clusters",
    "multi_window",
    "validated_levels",
    "zigzag",
    "mean_rev",
    "mean_rev_multi",
    "mean_rev_vol",
    "filter",
    "multi_filter",
    "filter_bidirectional",
    "filter_one",
    "trend_one",
    "filter_flat",
]

base_cfg = dict(
    symbol='XAUUSD', timeframe='H1', direction='both',
    train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    test_start=datetime(2021,1,1),  test_end=datetime(2022,1,1),
    search_type='lgmm', n_trials=500, n_models=1
)

configs = []
for lm in label_methods:
    cfg = base_cfg.copy()
    cfg["label_method"] = lm
    configs.append(cfg)

# Calcular núcleos por configuración
# total_cores = os.cpu_count()
# cores_per_config = max(1, total_cores // len(configs))

# Actualizar n_jobs en cada configuración
for cfg in configs:
    #cfg["n_jobs"] = cores_per_config
    cfg["tag"] = f"{cfg['symbol']} {cfg['timeframe']} {cfg['direction']} {cfg['label_method']} {cfg['search_type']} {cfg.get('search_subtype', '')}".strip()

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD H1 both filter lgmm] modelo 0 trial 1/500 ins=-1.000000 oos=-1.000000 avg=14.67s mem=405.36MB 
[XAUUSD H1 both mean_rev lgmm] modelo 0 trial 1/500 ins=-1.000000 oos=-1.000000 avg=22.63s mem=413.68MB 
[XAUUSD H1 both zigzag lgmm] modelo 0 trial 1/500 ins=-1.000000 oos=-1.000000 avg=23.32s mem=421.36MB 
[XAUUSD H1 both validated_levels lgmm] modelo 0 trial 1/500 ins=-1.000000 oos=-1.000000 avg=27.79s mem=416.15MB 
[XAUUSD H1 both trend_profit lgmm] modelo 0 trial 1/500 ins=-1.000000 oos=-1.000000 avg=28.46s mem=436.10MB 
[XAUUSD H1 both trend_one lgmm] modelo 0 trial 1/500 ins=0.316875 oos=0.018391 avg=32.01s mem=489.05MB 
[XAUUSD H1 both filter_one lgmm] modelo 0 trial 1/500 ins=-1.000000 oos=0.011238 avg=35.93s mem=468.47MB 
[XAUUSD H1 both mean_rev_vol lgmm] modelo 0 trial 1/500 ins=0.492237 oos=0.032074 avg=37.95s mem=506.91MB 
